In [2]:
#coding:utf-8
import numpy as np
# import math


class iteration():
    def __init__(self,n):
        self.A = None
        self.b = None
        self.x0 = None
        self.initialization(n)

    # 构造迭代矩阵An和初始值x0 和bn。
    def initialization(self, n):
        self.b = range(1, n + 1)
        self.A = []
        self.x0 = np.zeros((n,))
        for i in range(n):
            list = range(1, n + 1)
            list[i] = n ** 2
            self.A.append(list)
        self.A = np.array(self.A)

    #         return self.An,self.bn,self.x0

    # 由A的矩阵分解得到LU两个矩阵。
    def LU_decompose(self,A):
        length = len(A)
#         print length
        L = np.zeros((length, length))
        U = np.zeros((length, length))
        #     print L.dtype,U.dtype
        # 初始化向量
        for i in range(len(A)):
            U[0][i] = A[0][i]
            L[i][0] = A[i][0] * 1.0 / U[0][0]
        for r in range(1, len(A)):
            for i in range(r, len(A)):
                sum1 = 0
                sum2 = 0
                for k in range(0, r):
                    sum1 += L[r][k] * U[k][i]
                    sum2 += L[i][k] * U[k][r]
                U[r][i] = A[r][i] - sum1
                L[i][r] = (A[i][r] - sum2) * 1.0 / U[r][r]
        return L, U
    
    # 通过LU来求精确解
    def LU_solve(self):
        L,U = self.LU_decompose(self.A)
        length = len(L)
        y = np.array([0.0] * length)
        x = np.array([0.0] * length)
        # print x.dtype, y.dtype
        n = len(x) - 1
        #     print len(y)
        y[0] = self.b[0]
        for i in range(1, len(y)):
            sum1 = 0
            for k in range(0, i):
                sum1 += L[i, k] * y[k] * 1.0
            y[i] = self.b[i] - sum1
        x[n] = y[n] * 1.0 / U[n, n]
        for i in range(n - 1, -1, -1):
            sum2 = 0
            for k in range(i + 1, n + 1):
                sum2 += U[i, k] * x[k]
            x[i] = (y[i] - sum2) * 1.0 / U[i][i] * 1.0
        return  x
    
    # 雅可比迭代法
    def jacoby(self):
        x = self.LU_solve()
        n = len(self.b)
        x0=np.array([0.0]*n)
        x1 = np.array([0.0]*n)
        count = 0
        while(True):
            count +=1
            for i in range(0,n):
                sum1 =0
                for j in range(0,n):
                    if(j!=i):
                        sum1 +=self.A[i,j]*x0[j]
                x1[i] = (self.b[i]-sum1)*1.0/self.A[i,i]
            e = max(abs(x1-x))
            if(e<1.0/2*(10**(-4))):
                break
            else:
                x0 = x1
        return count,x1
    
    # 高斯-赛德尔迭代法
    def  Seidel(self):
        x = self.LU_solve()
        n = len(self.b)
        x0=np.array([0.0]*n)
        x1 = np.array([0.0]*n)
        count = 0
        while(True):
            count +=1
            for i in range(0,n):
                sum1 =0
                for j in range(0,n):
                    if(j<i):
                        sum1 +=self.A[i,j]*x1[j]
                    elif(j>i):
                        sum1 +=self.A[i,j]*x0[j]
                x1[i] = (self.b[i]-sum1)*1.0/self.A[i,i]
            e = max(abs(x1-x))
            if(e<1.0/2*(10**(-4))):
                break
            else:
                x0 = x1
        return count,x1
    
    # 超松弛迭代法，w代表的是松弛因子。
    def Sor(self,w):
        A = self.A
        b = self.b
        x = self.LU_solve()
        n = len(b)
        x0=np.array([0.0]*n)
        x1 = np.array([0.0]*n)
        count = 0
        while(True):
            count +=1
            for i in range(0,n):
                sum1 =0
                for j in range(0,n):
                    if(j<i):
                        sum1 +=A[i,j]*x1[j]
                    elif(j>=i):
                        sum1 +=A[i,j]*x0[j]
                x1[i] = x0[i]+w*1.0*(b[i]-sum1)*1.0/A[i,i]
            e = max(abs(x1-x))
            if(e<1.0/2*(10**(-4))):
                break
            else:
                x0 = x1
        return count,x1

In [7]:
print '当n为4时：'
it = iteration(4)
print '用LU来求得的精确解为：'
print 'x:',it.LU_solve()
print '\n用雅可比迭代法求得：'
count,x =it.jacoby()
print '迭代的次数是：',count,'x:',x
print '\n用高斯-赛德尔迭代法求得：'
count,x =it.Seidel()
print '迭代次数是：',count,'x:',x
print '\n用超松弛迭代法求得：'
wn = [0.6,0.8,1.0,1.2]
print '松弛因子 迭代次数        x'
for w in wn:
    count,x =it.Sor(w)
    print w,count,x

当n为4时：
用LU来求得的精确解为：
x: [-0.02271789  0.04708798  0.12763321  0.22160264]

用雅可比迭代法求得：
迭代的次数是： 5 x: [-0.02271687  0.04710775  0.12763533  0.22159971]

用高斯-赛德尔迭代法求得：
迭代次数是： 5 x: [-0.02270494  0.0470889   0.12762894  0.22160252]

用超松弛迭代法求得：
松弛因子 迭代次数        x
0.6 12 [-0.0227142   0.04710403  0.12766402  0.22157572]
0.8 8 [-0.02272497  0.0470859   0.12765072  0.22159499]
1.0 5 [-0.02270494  0.0470889   0.12762894  0.22160252]
1.2 7 [-0.02275042  0.04708037  0.12765342  0.22161188]


In [4]:
print '当n为8时：'
it = iteration(8)
print '用LU来求得的精确解为：'
print 'x:',it.LU_solve()
print '\n用雅可比迭代法求得：'
count,x =it.jacoby()
print '迭代的次数是：',count,'x:',x
print '\n用高斯-赛德尔迭代法求得：'
count,x =it.Seidel()
print '迭代次数是：',count,'x:',x
print '\n用超松弛迭代法求得：'
wn = [0.6,0.8,1.0,1.2]
print '松弛因子 迭代次数        x'
for w in wn:
    count,x =it.Sor(w)
    print w,count,x

当n为8时：
用LU来求得的精确解为：
x: [-0.01887378 -0.00304916  0.01329429  0.03018253  0.04764325  0.06570607
  0.08440266  0.103767  ]

用雅可比迭代法求得：
迭代的次数是： 5 x: [-0.0188907  -0.00306457  0.01328328  0.03017834  0.04764569  0.06571144
  0.08440589  0.10376747]

用高斯-赛德尔迭代法求得：
迭代次数是： 5 x: [-0.01887598 -0.00304952  0.01329617  0.03018593  0.04764641  0.06570741
  0.08440248  0.10376639]

用超松弛迭代法求得：
松弛因子 迭代次数        x
0.6 10 [-0.0188674  -0.00304108  0.01330492  0.03019597  0.04765851  0.06571948
  0.08440559  0.10374196]
0.8 6 [-0.01888595 -0.0030592   0.01328878  0.03018446  0.04765491  0.06572598
  0.08441916  0.10374647]
1.0 5 [-0.01887598 -0.00304952  0.01329617  0.03018593  0.04764641  0.06570741
  0.08440248  0.10376639]
1.2 7 [-0.01889702 -0.00307728  0.01326625  0.0301624   0.04763652  0.06571184
  0.08441407  0.10377589]


In [5]:
print '当n为16时：'
it = iteration(16)
print '用LU来求得的精确解为：'
print 'x:',it.LU_solve()
print '\n用雅可比迭代法求得：'
count,x =it.jacoby()
print '迭代的次数是：',count,'x:',x
print '\n高斯-赛德尔迭代法求得：'
count,x =it.Seidel()
print '迭代次数是：',count,'x:',x
print '\n超松弛迭代法求得：'
wn = [0.6,0.8,1.0,1.2]
print '松弛因子 迭代次数        x'
for w in wn:
    count,x =it.Sor(w)
    print w,count,x

当n为16时：
用LU来求得的精确解为：
x: [-0.01156394 -0.00767246 -0.00375021  0.00020316  0.00418803  0.00820478
  0.0122538   0.01633547  0.02045019  0.02459836  0.02878039  0.03299671
  0.03724772  0.04153387  0.04585559  0.05021332]

用雅可比迭代法求得：
迭代的次数是： 5 x: [-0.01157137 -0.0076802  -0.00375826  0.00019496  0.00417994  0.00819718
  0.01224711  0.01633008  0.02044639  0.02459629  0.02877995  0.03299757
  0.03724937  0.0415357   0.04585708  0.05021424]

高斯-赛德尔迭代法求得：
迭代次数是： 4 x: [-0.01154555 -0.00765258 -0.00372862  0.00022627  0.00421198  0.00822847
  0.01227586  0.01635447  0.02046486  0.02460788  0.02878459  0.03299621
  0.0372439   0.04152862  0.04585079  0.05020997]

超松弛迭代法求得：
松弛因子 迭代次数        x
0.6 8 [-0.01153876 -0.00764795 -0.00372594  0.00022754  0.00421272  0.00822983
  0.01227906  0.01636058  0.02047448  0.02462081  0.02879952  0.03301046
  0.03725335  0.04152776  0.04583309  0.05016854]
0.8 5 [-0.01157267 -0.0076807  -0.00375726  0.000198    0.00418543  0.00820537
  0.01225811  0.01634388  

In [6]:
print '当n为32时：'
it = iteration(32)
print '用LU来求得的精确解为：'
print 'x:',it.LU_solve()
print '\n用雅可比迭代法求得：'
count,x =it.jacoby()
print '迭代的次数是：',count,'x:',x
print '\n高斯-赛德尔迭代法求得：'
count,x =it.Seidel()
print '迭代次数是：',count,'x:',x
print '\n超松弛迭代法求得：'
wn = [0.6,0.8,1.0,1.2]
print '松弛因子 迭代次数        x'
for w in wn:
    count,x =it.Sor(w)
    print w,count,x

当n为32时：
用LU来求得的精确解为：
x: [-0.00634981 -0.00537754 -0.00440338 -0.0034273  -0.00244931 -0.0014694
 -0.00048756  0.00049621  0.00148192  0.00246958  0.00345918  0.00445074
  0.00544426  0.00643975  0.00743721  0.00843666  0.00943808  0.0104415
  0.01144691  0.01245433  0.01346376  0.0144752   0.01548866  0.01650415
  0.01752167  0.01854123  0.01956284  0.0205865   0.02161221  0.02263999
  0.02366984  0.02470176]

用雅可比迭代法求得：
迭代的次数是： 4 x: [-0.00638692 -0.00541344 -0.00443746 -0.00345901 -0.00247815 -0.00149494
 -0.00050944  0.00047826  0.0014681   0.00245997  0.00345379  0.00444946
  0.00544691  0.00644604  0.00744679  0.00844907  0.00945284  0.01045804
  0.01146465  0.01247266  0.01348206  0.01449289  0.01550517  0.01651899
  0.01753441  0.01855155  0.01957054  0.02059151  0.02161464  0.02264011
  0.02366812  0.02469888]

高斯-赛德尔迭代法求得：
迭代次数是： 4 x: [-0.00634428 -0.00537177 -0.00439724 -0.00342071 -0.0024422  -0.00146173
 -0.00047931  0.00050503  0.00149126  0.00247936  0.00346931  0.00446108